# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [10]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [11]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,college,64.8569,-147.8028,48.52,41,20,4.61,US,1712800589
1,1,isafjordur,66.0755,-23.1240,25.27,88,92,4.16,IS,1712800589
2,2,grytviken,-54.2811,-36.5092,34.12,95,32,14.61,GS,1712800590
3,3,vung tau,10.3460,107.0843,84.65,71,10,23.80,VN,1712800590
4,4,blackmans bay,-43.0167,147.3167,59.52,61,47,1.99,AU,1712800591


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [12]:
humidity = city_data_df["Humidity"].astype(int)

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriImagery",
    size = "Humidity",
    frame_width = 800,
    frame_height = 700, 
    scale = 0.5,
    color = "City"
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [13]:
# Narrow down cities that fit criteria and drop any results with null values

# A max temperature lower than 79 degrees but higher than 64
# Wind speed less than 10 mph
# Zero cloudiness
# Humidity less than 70%

min_temp_s = city_data_df["Max Temp"] > 62
max_temp_s = city_data_df["Max Temp"] < 79
wind_speed_s = city_data_df["Wind Speed"] < 10
cloud_s = city_data_df["Cloudiness"] == 0
humidity_s = city_data_df["Humidity"] < 70

selected_city_df = city_data_df[min_temp_s & max_temp_s & wind_speed_s & cloud_s & humidity_s]

# Drop any rows with null values
selected_city_df = selected_city_df.dropna()

# Display sample data
selected_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
150,150,tuxpan,19.5500,-103.4000,71.37,61,0,4.14,MX,1712800640
164,164,jackson,42.4165,-122.8345,75.07,30,0,4.61,US,1712800428
178,178,saint-pierre,-21.3393,55.4781,74.88,69,0,8.05,RE,1712800406
197,197,aswan,24.0934,32.9070,67.30,27,0,0.00,EG,1712800654
225,225,dubai,25.2582,55.3047,77.25,50,0,9.22,AE,1712800548
235,235,tianchang,37.9981,114.0156,75.92,31,0,5.93,CN,1712800666
293,293,crane,31.3974,-102.3501,64.94,25,0,9.69,US,1712800685
352,352,yuanping,38.7153,112.7575,63.52,46,0,3.11,CN,1712800703
367,367,siwa oasis,29.2041,25.5195,63.39,55,0,6.93,EG,1712800708
373,373,santa rosalia,27.3167,-112.2833,69.71,25,0,6.06,MX,1712800512


### Step 3: Create a new DataFrame called `hotel_df`.

In [14]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = selected_city_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
150,tuxpan,MX,19.5500,-103.4000,61,
164,jackson,US,42.4165,-122.8345,30,
178,saint-pierre,RE,-21.3393,55.4781,69,
197,aswan,EG,24.0934,32.9070,27,
225,dubai,AE,25.2582,55.3047,50,
235,tianchang,CN,37.9981,114.0156,31,
293,crane,US,31.3974,-102.3501,25,
352,yuanping,CN,38.7153,112.7575,46,
367,siwa oasis,EG,29.2041,25.5195,55,
373,santa rosalia,MX,27.3167,-112.2833,25,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [15]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "limit":20,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
tuxpan - nearest hotel: Hotel Ruiz
jackson - nearest hotel: Brookside
saint-pierre - nearest hotel: Tropic Hotel
aswan - nearest hotel: Oscar Hotel
dubai - nearest hotel: Holiday Inn Bur Dubai - Embassy District
tianchang - nearest hotel: No hotel found
crane - nearest hotel: No hotel found
yuanping - nearest hotel: No hotel found
siwa oasis - nearest hotel: فندق الكيلانى
santa rosalia - nearest hotel: Sol y Mar
salida - nearest hotel: Hampton Inn & Suites
vail - nearest hotel: Hampton Inn & Suites Tucson Tech Park
hollister - nearest hotel: Joshua Inn Bed & Breakfast


,City,Country,Lat,Lng,Humidity,Hotel Name
150,tuxpan,MX,19.5500,-103.4000,61,Hotel Ruiz
164,jackson,US,42.4165,-122.8345,30,Brookside
178,saint-pierre,RE,-21.3393,55.4781,69,Tropic Hotel
197,aswan,EG,24.0934,32.9070,27,Oscar Hotel
225,dubai,AE,25.2582,55.3047,50,Holiday Inn Bur Dubai - Embassy District
235,tianchang,CN,37.9981,114.0156,31,No hotel found
293,crane,US,31.3974,-102.3501,25,No hotel found
352,yuanping,CN,38.7153,112.7575,46,No hotel found
367,siwa oasis,EG,29.2041,25.5195,55,فندق الكيلانى
373,santa rosalia,MX,27.3167,-112.2833,25,Sol y Mar


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [16]:
# Configure the map plot
selected_hotel_df = hotel_df[hotel_df['Hotel Name'] != "No hotel found"]
map_plot_2 = selected_hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=800,
    frame_height=700,
    size="Humidity",
    scale=1.0,
    color="City",
    hover_cols=['Hotel Name', 'City', 'Country']
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)